# trajectory_analyze

particle trajectory analysis

file requirements:
- particle netcdf file generated by parcels after simulation
- the actual ocean data the particles ran on

In [ ]:
%matplotlib inline

In [ ]:
import sys
from datetime import timedelta, datetime
import math
import xarray as xr
import numpy as np
from parcels import FieldSet, ParticleSet, JITParticle
import matplotlib.pyplot as plt

from utils import xr_dataset_to_fieldset, conv_to_dataarray, generate_mask, show_particles

In [ ]:
# ocean current netcdf data the particles used
d_nc = "west_coast_1km_hourly_netcdfs/west_coast_1km_hourly_region_tijuana_river_lin_aggr.nc"
# the data generated by ParticleFile
p_nc = "particledata/particle_tijuana_less.nc"

In [ ]:
d_ds = xr.open_dataset(d_nc)
p_ds = xr.open_dataset(p_nc)
fs = xr_dataset_to_fieldset(d_ds)
d_ds.close()
p_ds.close()

In [ ]:
d_ds

In [ ]:
p_ds

In [ ]:
d_mask = conv_to_dataarray(generate_mask(d_ds["u"].values), d_ds["u"])

### check whether a particle collided with land

conditions:
- only check the latest position of a particle before it's deleted or stuck on land or something
- particle is on a position where even after fieldset interpolation, vector values are still zero

however, there may be some regions of actual ocean that just don't have data, so it's not foolproof

In [ ]:
def time_to_str(time):
    """
    Args:
        time (np.datetime64)
    """
    return str(time).split(".")[0]

### some immediate problems

- particles are out of bounds even though I use coordinate values on the SAME FIELDSET during simulation
    - ??????????????? how


In [ ]:
collided_land = np.zeros(p_ds["trajectory"].shape, dtype=bool)
lat_ls = []
lon_ls = []
for p_iter in range(len(p_ds["trajectory"])):
    p = p_ds["trajectory"][p_iter]
    # find the first state of the particle where it got stuck on land something
    for step in range(len(p)):
        if np.isnan(p[step]):
            break
        time = p_ds["time"].values[p_iter, step]
        time_secs = p_ds["lifetime"].values[p_iter, step] + p_ds["spawntime"].values[p_iter, step]
        lat = p_ds["lat"].values[p_iter, step]
        lon = p_ds["lon"].values[p_iter, step]
        total_vel = abs(fs.U[time_secs, 0, lat, lon]) + abs(fs.V[time_secs, 0, lat, lon])
        # first, check if the vector speed at that coordinate is 0 after interpolation
        # second, check if the particle is not in an invalid coordinate (a coordinate supposed to have data)
        if total_vel <= sys.float_info.epsilon and not d_mask.sel(time=time, lat=lat, lon=lon, method="nearest"):
            lat_ls.append(lat)
            lon_ls.append(lon)
            print(f"particle {p_iter} beached at time {time_to_str(time)}")
            print(f"    spawn time: {time_to_str(p_ds['time'].values[p_iter, 0])}")
            print(f"    spawn coord: ({p_ds['lat'].values[p_iter, 0]}, {p_ds['lon'].values[p_iter, 0]})")
            print(f"    lifetime: {p_ds['lifetime'].values[p_iter, step]}")
            break

In [ ]:
show_particles(fs, lat_ls, lon_ls)